In [1]:
!pip install torchcam
import torch
import torchvision
from torch import nn
import numpy as np
from tqdm import tqdm
from torchvision.datasets import CIFAR10
from torchvision.models import resnet
from torch.utils.data import DataLoader
from torchvision import transforms
from datetime import datetime
from google.colab import drive
import random
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import os.path
import pickle
from typing import Any, Callable, Optional, Tuple

import numpy as np
from PIL import Image
imagetotensor= transforms.ToTensor()
tensortopil = transforms.ToPILImage()
def sample(tenss,number = 1):
    #Supposed to return index based on weighted random choice
    #Issue here is although code is likely to work gonna be slower than prebuilt options
    output = []
    value = np.random.uniform(size = number)
    for i in range(4):
        for j in range(4):
            value -= int(tenss[i,j])
            for k in range(len(value)):
              if value[k]<=0:
                output.append((i,j))

              np.delete(value,np.where(value<=0))
    if len(output)>1:
      return output

    return [(3,3)]
finaldata = torch.ones(50000,4,4)
finaldata=torch.load('/content/drive/My Drive/Models/Supervised/cutdata.pt')
comparator = torch.ones(4,4)
for i in range(50000):
  if torch.all(torch.eq(comparator,finaldata[i])):
    print(i)
    k+=1
  finaldata[i] = finaldata[i]/torch.sum(finaldata[i])

class indicesCIFAR10(CIFAR10):
  #Not only changes so indices are added, also now cutouts pre transform the image wait actually since pretransformation cutout, no longer need return indices or edit in post
    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img, target = self.data[index], self.targets[index]
        img = Image.fromarray(img)
        if self.train==True:
          
          img, target = self.data[index], self.targets[index]
          img = imagetotensor(img)
          xsys = sample(finaldata[index],3)
          for x,y in xsys:
            img[:,x*8:x*8+8,y*8:8*y+8] = 0
          img = tensortopil(img)
        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
      
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target,index

batch_size = 128
train_transform = transforms.Compose([
    
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])



train_data = CIFAR10(root='data', train=True, transform=train_transform, download=True)


def get_indices(dataset, num_samples):
  indices = []
  for Class in range(10):
    for j in range(num_samples):
      x = np.random.randint(0, 50000)
      while dataset.targets[x] != Class:
        x = np.random.randint(0, 50000)
      indices.append(x)
  return indices


train_data = indicesCIFAR10(root='data', train=True, transform=train_transform, download=True)
test_data = indicesCIFAR10(root='data', train=False, transform=test_transform, download=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=16, pin_memory=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [6]:

class ModelBase(nn.Module):
    """
    Common CIFAR ResNet recipe.
    Comparing with ImageNet ResNet recipe, it:
    (i) replaces conv1 with kernel=3, str=1
    (ii) removes pool1
    """
    def __init__(self, feature_dim=10, arch='resnet18', bn_splits=16):
        super(ModelBase, self).__init__()

        # use split batchnorm
        norm_layer = nn.BatchNorm2d
        # get specified resnet model
        resnet_arch = getattr(resnet, arch)
        net = resnet_arch(num_classes=feature_dim, norm_layer=norm_layer)

        # make changes to original resnet
        self.net = []
        for name, module in net.named_children():
            if name == 'conv1':
                module = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
            if isinstance(module, nn.MaxPool2d):
                continue
            if isinstance(module, nn.Linear):
                self.net.append(nn.Flatten(1))
            self.net.append(module)

        # build net
        self.net = nn.Sequential(*self.net)

    def forward(self, x):
        x = self.net(x)
        # note: not normalized here
        return x

def train_val(net, data_loader, train_optimizer):
    global lr
    schedule = [60, 80]
    criterion = nn.CrossEntropyLoss().cuda()
    is_train = train_optimizer is not None
    net.train() if is_train else net.eval()

    total_loss, total_correct_1, total_correct_3, total_num, data_bar = 0.0, 0.0, 0.0, 0, tqdm(data_loader, position=0, leave=True)
    with (torch.enable_grad() if is_train else torch.no_grad()):
        for data, target in data_bar:
            #print(target.size())
            #insert cropping here
            data, target = data.cuda(non_blocking=True), target.cuda(non_blocking=True)
            out = net(data)
            loss = criterion(out, target)

            if is_train:
                train_optimizer.zero_grad()
                loss.backward()
                train_optimizer.step()


            total_num += data.size(0)
            total_loss += loss.item() * data.size(0)
            prediction = torch.argsort(out, dim=-1, descending=True)
            total_correct_1 += torch.sum((prediction[:, 0:1] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()
            total_correct_3 += torch.sum((prediction[:, 0:3] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()
            
            data_bar.set_description('{} Epoch: [{}/{}] lr: {:.4f} Loss: {:.4f} ACC@1: {:.2f}% ACC@5: {:.2f}%'
                                     .format('Train' if is_train else 'Test', epoch, epochs, lr, total_loss / total_num,
                                             total_correct_1 / total_num * 100, total_correct_3 / total_num * 100))
        if is_train:  
          if schedule is not None:
            for milestone in schedule:
                lr *= 0.1 if epoch == milestone else 1.
            for param_group in train_optimizer.param_groups:
                param_group['lr'] = lr

        return total_correct_1/total_num * 100


In [ ]:
# torch.save(finaldata, '/content/drive/My Drive/Models/Supervised/cutdata.pt')
# finaldata = torch.ones(50000,4,4)
# finaldata=torch.load('/content/drive/My Drive/Models/Supervised/cutdata.pt')
# for i in range(50000):
#     finaldata[i] = finaldata[i]/torch.sum(finaldata[i])


In [ ]:
def sample(tenss):
    #Supposed to return index based on weighted random choice
    #Issue here is although code is likely to work gonna be slower than prebuilt options
    value = np.random.uniform()
    for i in range(4):
        for j in range(4):
            value -= tenss[i,j]
            if value<=0:
                return i,j
    return 3,3
sample(finaldata[6893])

tensor([[5.0653e-05, 1.7633e-02, 2.7128e-02, 5.9558e-02],
        [0.0000e+00, 4.2108e-02, 9.9771e-02, 6.2544e-02],
        [9.6054e-03, 6.9857e-02, 1.9324e-01, 9.7627e-02],
        [1.4498e-02, 9.5292e-02, 1.0997e-01, 1.0112e-01]])


In [ ]:
indices = get_indices(train_data, 1000)
sampler = torch.utils.data.SubsetRandomSampler(indices)
supervised = ModelBase()

supervised.load_state_dict(torch.load("/content/drive/My Drive/Models/Supervised/supervisedresnet18v1.1.pth"))

AttributeError: ignored

In [ ]:
indices = get_indices(train_data, 1000)
sampler = torch.utils.data.SubsetRandomSampler(indices)
train_loader = DataLoader(train_data, batch_size=batch_size, sampler = sampler , num_workers=16, pin_memory=True)
for batch in train_loader:
  data,target = batch
  special = data[0]
  break

In [ ]:
from torchcam.methods import SmoothGradCAMpp
supervised = ModelBase()

supervised.load_state_dict(torch.load("/content/drive/My Drive/Models/Supervised/supervisedresnet18v1.1.pth"))
cam_extractor = SmoothGradCAMpp(supervised)
data = special
out = supervised(data.unsqueeze(0))
print(out)
activation_map = cam_extractor(out.squeeze(0).argmax().item(), out)
print(activation_map)
#result = overlay_mask(to_pil_image(data), to_pil_image(activation_map[0].squeeze(0), mode='F'), alpha=0.5)
# Display it
#plt.imshow(result); plt.axis('off'); plt.tight_layout(); plt.show()


tensor([[ -6.4669,  18.5649,  -1.9640,  -6.8271, -12.8749,  -8.3182,  -1.3064,
         -10.8043,  -7.3363,  -0.1034]], grad_fn=<AddmmBackward0>)
[tensor([[[0.2195, 0.4906, 0.5149, 0.2337],
         [0.2724, 0.7078, 1.0000, 0.6716],
         [0.0936, 0.5853, 0.9919, 0.8324],
         [0.0000, 0.1363, 0.5818, 0.4546]]])]


In [ ]:
print(cam_extractor._input.shape)
print(activation_map)

torch.Size([1, 3, 32, 32])
[tensor([[[0.0451, 0.0551, 0.0421, 0.0000],
         [0.1042, 0.3766, 0.4040, 0.3928],
         [0.3212, 0.5861, 0.7993, 1.0000],
         [0.1716, 0.3910, 0.6594, 0.8727]]])]


In [10]:
import random
from torchvision.io.image import read_image
from torchvision.transforms.functional import normalize, resize, to_pil_image
from torchvision.models import resnet18
from torchcam.methods import GradCAMpp

from time import time 
import matplotlib.pyplot as plt
from torchcam.utils import overlay_mask
from typing import Any, Callable, Optional, Tuple

import numpy as np
from PIL import Image

def train_val(net, data_loader, train_optimizer):
    global lr
    schedule = [60, 80]
    criterion = nn.CrossEntropyLoss().cuda()
    is_train = train_optimizer is not None
    net.train() if is_train else net.eval()
    sa = 0
    total_loss, total_correct_1, total_correct_3, total_num, data_bar = 0.0, 0.0, 0.0, 0, tqdm(data_loader, position=0, leave=True)
    with (torch.enable_grad() if is_train else torch.no_grad()):
        for data, target,indexes in data_bar:
            data, target = data.cuda(non_blocking=True), target.cuda(non_blocking=True)
            out = net(data)
            loss = criterion(out, target)

            if is_train:
                train_optimizer.zero_grad()
                loss.backward()
                train_optimizer.step()


            total_num += data.size(0)
            total_loss += loss.item() * data.size(0)
            prediction = torch.argsort(out, dim=-1, descending=True)
            total_correct_1 += torch.sum((prediction[:, 0:1] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()
            total_correct_3 += torch.sum((prediction[:, 0:3] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()
            
            data_bar.set_description('{} Epoch: [{}/{}] lr: {:.4f} Loss: {:.4f} ACC@1: {:.2f}% ACC@5: {:.2f}%'
                                     .format('Train' if is_train else 'Test', epoch, epochs, lr, total_loss / total_num,
                                             total_correct_1 / total_num * 100, total_correct_3 / total_num * 100))
        if is_train:  
          if schedule is not None:
            for milestone in schedule:
                lr *= 0.1 if epoch == milestone else 1.
            for param_group in train_optimizer.param_groups:
                param_group['lr'] = lr

        return total_correct_1/total_num * 100

train_acc = []
test_acc = []
for i in range(1):
  print(f"Random Seed: {i}")
  best_train = 0
  best_test = 0
  np.random.seed(i)

  indices = get_indices(train_data, 1000)
  sampler = torch.utils.data.SubsetRandomSampler(indices)

  train_loader = DataLoader(train_data, batch_size=batch_size, sampler = sampler , num_workers=16, pin_memory=True)
  resnet18 = ModelBase()
  resnet18.cuda()
  lr = 1e-3
  optimizer = torch.optim.Adam(resnet18.parameters(), lr=lr, weight_decay = 1e-5)
  epoch_start = 1
  epochs = 100
  for epoch in range(epoch_start, epochs+1):
    acc1 = train_val(resnet18, train_loader, optimizer)
    acc2 = train_val(resnet18, test_loader, None)
    if acc1 > best_train:
      best_train = acc1
    if acc2 > best_test:
      best_test = acc2
  train_acc.append(acc1)
  test_acc.append(acc2)
  
  torch.save(resnet18.state_dict(), "/content/drive/My Drive/Models/Supervised/CAMsupervisedsample3.pth")
print(train_acc)
print(test_acc)
import numpy as np
train_acc = np.array(train_acc)
test_acc = np.array(test_acc)
print(np.mean(train_acc))
print(np.mean(test_acc))
print(np.std(train_acc))
print(np.std(test_acc))


Random Seed: 0


Train Epoch: [1/100] lr: 0.0010 Loss: 1.9332 ACC@1: 28.84% ACC@5: 63.01%: 100%|██████████| 79/79 [00:19<00:00,  4.03it/s]
Test Epoch: [1/100] lr: 0.0010 Loss: 1.7366 ACC@1: 32.94% ACC@5: 72.98%: 100%|██████████| 79/79 [00:05<00:00, 14.93it/s]
Train Epoch: [2/100] lr: 0.0010 Loss: 1.7484 ACC@1: 35.63% ACC@5: 70.41%: 100%|██████████| 79/79 [00:18<00:00,  4.16it/s]
Test Epoch: [2/100] lr: 0.0010 Loss: 1.5850 ACC@1: 40.49% ACC@5: 77.06%: 100%|██████████| 79/79 [00:04<00:00, 16.00it/s]
Train Epoch: [3/100] lr: 0.0010 Loss: 1.6305 ACC@1: 40.50% ACC@5: 74.23%: 100%|██████████| 79/79 [00:20<00:00,  3.82it/s]
Test Epoch: [3/100] lr: 0.0010 Loss: 1.5579 ACC@1: 44.61% ACC@5: 77.08%: 100%|██████████| 79/79 [00:04<00:00, 16.18it/s]
Train Epoch: [4/100] lr: 0.0010 Loss: 1.5606 ACC@1: 43.51% ACC@5: 76.60%: 100%|██████████| 79/79 [00:18<00:00,  4.20it/s]
Test Epoch: [4/100] lr: 0.0010 Loss: 1.4754 ACC@1: 47.02% ACC@5: 79.83%: 100%|██████████| 79/79 [00:04<00:00, 16.00it/s]
Train Epoch: [5/100] lr: 0.0

[88.46000000000001]
[82.85]
88.46000000000001
82.85
0.0
0.0


In [11]:
class ModelBase(nn.Module):
    """
    Common CIFAR ResNet recipe.
    Comparing with ImageNet ResNet recipe, it:
    (i) replaces conv1 with kernel=3, str=1
    (ii) removes pool1
    """
    def __init__(self, feature_dim=10, arch='resnet18', bn_splits=16):
        super(ModelBase, self).__init__()

        # use split batchnorm
        norm_layer = nn.BatchNorm2d
        # get specified resnet model
        resnet_arch = getattr(resnet, arch)
        net = resnet_arch(num_classes=feature_dim, norm_layer=norm_layer)

        # make changes to original resnet
        self.net = []
        for name, module in net.named_children():
            if name == 'conv1':
                module = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
            if isinstance(module, nn.MaxPool2d):
                continue
            if isinstance(module, nn.Linear):
                self.net.append(nn.Flatten(1))
            self.net.append(module)

        # build net
        self.net = nn.Sequential(*self.net)

    def forward(self, x):
        x = self.net(x)
        x = torch.flatten(x,start_dim=1)
        # note: not normalized here
        return x
class classifier(nn.Module):

    def __init__(self, feature_dim=10, arch='resnet18', bn_splits=16):
        super(classifier, self).__init__()

        self.net = nn.Linear(1024,10)

    def forward(self, x):
        x = self.net(x)
        # note: not normalized here
        return x
supervised = ModelBase()
supervised.load_state_dict(torch.load("/content/drive/My Drive/Models/Supervised/supervisedresnet18v1.1.pth"))
cutout = ModelBase()
cutout.load_state_dict(torch.load("/content/drive/My Drive/Models/Supervised/CAMsupervisedsample3.pth"))
supervised.net =supervised.net[:-1]
cutout.net =cutout.net[:-1]
for i in supervised.parameters():
  i.requires_grad = False
for i in cutout.parameters():
  i.requires_grad = False
supervised=supervised.cuda()
cutout = cutout.cuda()
finallayer = classifier()

In [ ]:



def train_val(base1,base2,final, data_loader, train_optimizer):
    global lr
    schedule = [60, 80]
    criterion = nn.CrossEntropyLoss().cuda()
    is_train = train_optimizer is not None
    final.train() if is_train else final.eval()
    sa = 0
    total_loss, total_correct_1, total_correct_3, total_num, data_bar = 0.0, 0.0, 0.0, 0, tqdm(data_loader, position=0, leave=True)
    with (torch.enable_grad() if is_train else torch.no_grad()):
        for data, target,indexes in data_bar:
            data = data.cuda()
            output1 = base1(data)
            output2 = base2(data)
            #print(output1.shape,output2.shape)
            data = torch.cat((output1,output2),1)
            #print(data.shape)

            data, target = data.cuda(non_blocking=True), target.cuda(non_blocking=True)
            out = final(data)
            loss = criterion(out, target)

            if is_train:
                train_optimizer.zero_grad()
                loss.backward()
                train_optimizer.step()


            total_num += data.size(0)
            total_loss += loss.item() * data.size(0)
            prediction = torch.argsort(out, dim=-1, descending=True)
            total_correct_1 += torch.sum((prediction[:, 0:1] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()
            total_correct_3 += torch.sum((prediction[:, 0:3] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()
            
            data_bar.set_description('{} Epoch: [{}/{}] lr: {:.4f} Loss: {:.4f} ACC@1: {:.2f}% ACC@5: {:.2f}%'
                                     .format('Train' if is_train else 'Test', epoch, epochs, lr, total_loss / total_num,
                                             total_correct_1 / total_num * 100, total_correct_3 / total_num * 100))
        if is_train:  
          if schedule is not None:
            for milestone in schedule:
                lr *= 0.1 if epoch == milestone else 1.
            for param_group in train_optimizer.param_groups:
                param_group['lr'] = lr

        return total_correct_1/total_num * 100

train_acc = []
test_acc = []
for i in range(1):
  print(f"Random Seed: {i}")
  best_train = 0
  best_test = 0
  np.random.seed(i)

  indices = get_indices(train_data, 1000)
  sampler = torch.utils.data.SubsetRandomSampler(indices)

  train_loader = DataLoader(train_data, batch_size=batch_size, sampler = sampler , num_workers=16, pin_memory=True)
  finallayer = classifier()
  finallayer=finallayer.cuda()
  lr = 1e-4
  optimizer = torch.optim.Adam(finallayer.parameters(), lr=lr, weight_decay = 1e-5)
  epoch_start = 1
  epochs = 100
  for epoch in range(epoch_start, epochs+1):
    acc1 = train_val(supervised,cutout,finallayer, train_loader, optimizer)
    acc2 = train_val(supervised,cutout,finallayer, test_loader, None)
    if acc1 > best_train:
      best_train = acc1
    if acc2 > best_test:
      best_test = acc2
  train_acc.append(acc1)
  test_acc.append(acc2)
  
  torch.save(finallayer.state_dict(), "/content/drive/My Drive/Models/Supervised/finallayerconcat1024butCAM.pth")
print(train_acc)
print(test_acc)
top10 = np.sort(train_acc)[::-1]
print("top10 trainacc",np.mean(top10[:10]))
top10 = np.sort(test_acc)[::-1]
print("top10 testacc",np.mean(top10[:10]))
print("best test",np.sort(test_acc)[::-1][0])
train_acc = np.array(train_acc)
test_acc = np.array(test_acc)
print(np.mean(train_acc))
print(np.mean(test_acc))
print(np.std(train_acc))
print(np.std(test_acc))


Random Seed: 0


Train Epoch: [1/100] lr: 0.0001 Loss: 1.5209 ACC@1: 63.60% ACC@5: 84.75%: 100%|██████████| 79/79 [00:17<00:00,  4.56it/s]
Test Epoch: [1/100] lr: 0.0001 Loss: 0.9562 ACC@1: 80.75% ACC@5: 95.78%: 100%|██████████| 79/79 [00:07<00:00, 10.40it/s]
Train Epoch: [2/100] lr: 0.0001 Loss: 0.7774 ACC@1: 86.75% ACC@5: 96.88%: 100%|██████████| 79/79 [00:17<00:00,  4.63it/s]
Test Epoch: [2/100] lr: 0.0001 Loss: 0.6829 ACC@1: 82.51% ACC@5: 96.44%: 100%|██████████| 79/79 [00:07<00:00, 10.36it/s]
Train Epoch: [3/100] lr: 0.0001 Loss: 0.5874 ACC@1: 87.96% ACC@5: 97.07%: 100%|██████████| 79/79 [00:16<00:00,  4.75it/s]
Test Epoch: [3/100] lr: 0.0001 Loss: 0.5905 ACC@1: 82.85% ACC@5: 96.49%: 100%|██████████| 79/79 [00:07<00:00, 10.53it/s]
Train Epoch: [4/100] lr: 0.0001 Loss: 0.4975 ACC@1: 88.47% ACC@5: 97.03%: 100%|██████████| 79/79 [00:16<00:00,  4.76it/s]
Test Epoch: [4/100] lr: 0.0001 Loss: 0.5487 ACC@1: 82.89% ACC@5: 96.55%: 100%|██████████| 79/79 [00:07<00:00, 10.54it/s]
Train Epoch: [5/100] lr: 0.0

In [ ]:
#This is originally to get the probabilities of places to cut out, however now no longer needed as results have been saved
from torchcam.methods import GradCAMpp
class ModelBase(nn.Module):
    """
    Common CIFAR ResNet recipe.
    Comparing with ImageNet ResNet recipe, it:
    (i) replaces conv1 with kernel=3, str=1
    (ii) removes pool1
    """
    def __init__(self, feature_dim=10, arch='resnet18', bn_splits=16):
        super(ModelBase, self).__init__()

        # use split batchnorm
        norm_layer = nn.BatchNorm2d
        # get specified resnet model
        resnet_arch = getattr(resnet, arch)
        net = resnet_arch(num_classes=feature_dim, norm_layer=norm_layer)

        # make changes to original resnet
        self.net = []
        for name, module in net.named_children():
            if name == 'conv1':
                module = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
            if isinstance(module, nn.MaxPool2d):
                continue
            if isinstance(module, nn.Linear):
                self.net.append(nn.Flatten(1))
            self.net.append(module)

        # build net
        self.net = nn.Sequential(*self.net)

    def forward(self, x):
        x = self.net(x)
        # note: not normalized here
        return x
#Pretransform since getitem transform is very costly on time
tempdata = []
totensor = transforms.ToTensor()
train_data = CIFAR10(root='data', train=True, transform=test_transform, download=True)
print(type(train_data.data[0]))
for i in range(50000):
  tempdata.append(totensor(train_data.data[i]).unsqueeze(0))
print(len(tempdata))
tempdata = torch.cat(tempdata)
finaldata = torch.ones(50000,4,4)
finaldata =torch.load('/content/drive/My Drive/Models/Supervised/cutdatanotransform.pt')
print(tempdata.shape)
supervised = ModelBase()
supervised.load_state_dict(torch.load("/content/drive/My Drive/Models/Supervised/supervisedresnet18v1.1.pth"))
cam_extractor = GradCAMpp(supervised)
for index in range(21000,50000,1000):
  data = tempdata[index:index+1000]
  target = train_data.targets[index:index+1000]
  pair = []

  actualinput = data.clone()
  classes =[]
  classind = []
  for i in range(10):
    classes.append([])
    classind.append([])
  for i in range(len(data)):
    classes[target[i]].append(data[i].unsqueeze(0))
    classind[target[i]].append(i)
  for i in range(10):
    classes[i] = torch.cat(classes[i])
  #print(classind)
  for i in range(10):
    data = classes[i]
    out = supervised(data)
    activation_map = cam_extractor(scores=out,class_idx=i)
    activation_map = activation_map[0]
    for id in range(len(classind[i])):
      #argpos = (activation_map[id]==torch.max(activation_map[id])).nonzero()[0]
      finaldata[index+classind[i][id]] = activation_map[id]
      #actualinput[classind[i+index],:,argpos[0]*8:argpos[0]*8+8,argpos[1]*8:argpos[1]*8+8]=0
  torch.save(finaldata, '/content/drive/My Drive/Models/Supervised/cutdatanotransform.pt')
  print(index)


Files already downloaded and verified
<class 'numpy.ndarray'>
50000
torch.Size([50000, 3, 32, 32])


21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000


In [ ]:
temp = torch.ones(4,4)
print(temp,finaldata[0])
for i in range(50000):
  if torch.all(torch.eq(temp,finaldata[i])):
    print(i)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]]) tensor([[0.0000, 0.3266, 0.0300, 0.2849],
        [0.0948, 0.3479, 0.7162, 0.5849],
        [0.3012, 0.6110, 0.9452, 0.5517],
        [0.5401, 0.6715, 0.9870, 1.0000]])


In [ ]:
from torchvision.io.image import read_image
from torchvision.transforms.functional import normalize, resize, to_pil_image
from torchvision.models import resnet18
from torchcam.methods import GradCAMpp

from time import time 
import matplotlib.pyplot as plt
from torchcam.utils import overlay_mask

# for i in supervised.parameters():
#   i.requires_grad=False
indices = get_indices(train_data, 1000)
sampler = torch.utils.data.SubsetRandomSampler(indices)
supervised = ModelBase()
supervised.load_state_dict(torch.load("/content/drive/My Drive/Models/Supervised/supervisedresnet18v1.1.pth"))
supervised.eval()
cam_extractor = GradCAMpp(supervised)
batchno = 0
st = time()
train_loader = DataLoader(train_data, batch_size=
                          batch_size, sampler = sampler , num_workers=16, pin_memory=True)
for batch in train_loader:
  pair = []
  data,target = batch
  actualinput = data.clone()
  classes =[]
  classind = []
  for i in range(10):
    classes.append([])
    classind.append([])
  for i in range(len(data)):
    classes[target[i]].append(data[i].unsqueeze(0))
    classind[target[i]].append(i)
  for i in range(10):
    classes[i] = torch.cat(classes[i])
  for i in range(10):
    data = classes[i]
    out = supervised(data)
    activation_map = cam_extractor(scores=out,class_idx=9)
    activation_map = activation_map[0]
    for id in range(len(classind[i])):
      argpos = (activation_map[id]==torch.max(activation_map[id])).nonzero()[0]
      actualinput[classind[i],:,argpos[0]*8:argpos[0]*8+8,argpos[1]*8:argpos[1]*8+8]=0
    data = actualinput

et=time()
print(batchno,et-st)  


torch.Size([1, 3, 32, 32])
torch.Size([16, 3, 32, 32])
torch.Size([16, 10]) torch.Size([16, 3, 32, 32]) tensor(130)
torch.Size([15, 10]) torch.Size([15, 3, 32, 32]) tensor(31)
torch.Size([13, 10]) torch.Size([13, 3, 32, 32]) tensor(52)
torch.Size([20, 10]) torch.Size([20, 3, 32, 32]) tensor(3)
torch.Size([13, 10]) torch.Size([13, 3, 32, 32]) tensor(114)
torch.Size([12, 10]) torch.Size([12, 3, 32, 32]) tensor(45)
torch.Size([5, 10]) torch.Size([5, 3, 32, 32]) tensor(26)
torch.Size([10, 10]) torch.Size([10, 3, 32, 32]) tensor(47)
torch.Size([10, 10]) torch.Size([10, 3, 32, 32]) tensor(28)
torch.Size([14, 10]) torch.Size([14, 3, 32, 32]) tensor(129)
0 10.8184072971344
